# Notebook 7/29, starting to summarize for thesis

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['SPS_HOME'] = "/global/homes/e/eramey16/fsps"
from docker import classify
from docker import db_util as util

import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
classy = classify.Classifier()
unlensed_gals = classy.query_galaxy(ra=150, dec=2, radius=5, limit=None)


KeyboardInterrupt



In [ ]:
lrgs = util.clean_desi(unlensed_gals)
g, r, z = lrgs.dered_mag_g, lrgs.dered_mag_r, lrgs.dered_mag_z
w1, w2 = lrgs.dered_mag_w1, lrgs.dered_mag_w2
cut1 = (z - w1) > (0.8 * (r - z) - 0.6)
cut2 = ((g - w1) > 2.9) | ((r - w1) > 1.8)
cut3 = (((r-w1) > 1.8*(w1-17.14)) & ((r-w1) > (w1-16.33))) | ((r-w1) > 3.3)
print(len(lrgs[cut1 & cut2 & cut3]), "out of", len(lrgs)), "gals"
lrgs = lrgs[cut1 & cut2 & cut3]
lrgs['ls_id'] = lrgs['ls_id'].astype(int)
# path = os.path.expandvars("$SCRATCH/data/monocle/LRGs")
# lrgs[['ls_id']].to_csv(f"{path}/lensed_LRGs.dat", index=False)
z_unlensed = lrgs['z_phot_median']

In [ ]:
_ = plt.hist(unlensed_gals['z_phot_median'], bins=100)

In [ ]:
with classy.engine.connect() as conn:
    gal_tbl = sa.Table("lrg_train", classy.meta, autoload_with=classy.engine)
    stmt = sa.select(gal_tbl)
    lensed_gals = pd.DataFrame(conn.execute(stmt))
z_lensed = lensed_gals['z_phot_median']
_ = plt.hist(z_lensed, bins=100)
len(z_lensed)

In [ ]:
# Select galaxies from the same redshift distribution as lensed galaxies (assuming gaussian)
# i.e. read lensed LRG table in and plot the redshift distribution
def create_matched_unlensed_sample(unlensed_df, lensed_redshifts, n_sample=3000):
    # Use more bins for finer matching
    bins = np.linspace(lensed_redshifts.min(), lensed_redshifts.max(), 25)
    
    # Get target distribution
    target_hist, _ = np.histogram(lensed_redshifts, bins=bins)
    target_proportions = target_hist / target_hist.sum()
    
    # Sample proportionally from each bin
    sampled_indices = []
    for i in range(len(bins)-1):
        bin_mask = (unlensed_df['z_phot_median'] >= bins[i]) & (unlensed_df['z_phot_median'] < bins[i+1])
        bin_candidates = unlensed_df[bin_mask].index
        
        n_needed = int(target_proportions[i] * n_sample)
        if len(bin_candidates) >= n_needed:
            sampled_indices.extend(np.random.choice(bin_candidates, n_needed, replace=False))
    
    return unlensed_df.loc[sampled_indices]

def create_2d_matched_sample(unlensed_df, lensed_df, n_sample=3000):
    # Create 2D bins for magnitude and redshift
    mag_bins = np.linspace(lensed_df['dered_mag_r'].min(), lensed_df['dered_mag_r'].max(), 15)
    z_bins = np.linspace(lensed_df['z_phot_median'].min(), lensed_df['z_phot_median'].max(), 15)
    
    # Get 2D histogram of lensed sample
    lensed_hist, _, _ = np.histogram2d(lensed_df['dered_mag_r'], lensed_df['z_phot_median'], 
                                      bins=[mag_bins, z_bins])
    target_proportions = lensed_hist / lensed_hist.sum()
    
    sampled_indices = []
    for i in range(len(mag_bins)-1):
        for j in range(len(z_bins)-1):
            # Find candidates in this (magnitude, redshift) bin
            mag_mask = (unlensed_df['dered_mag_r'] >= mag_bins[i]) & (unlensed_df['dered_mag_r'] < mag_bins[i+1])
            z_mask = (unlensed_df['z_phot_median'] >= z_bins[j]) & (unlensed_df['z_phot_median'] < z_bins[j+1])
            bin_candidates = unlensed_df[mag_mask & z_mask].index
            
            n_needed = int(target_proportions[i,j] * n_sample)
            if len(bin_candidates) >= n_needed and n_needed > 0:
                sampled_indices.extend(np.random.choice(bin_candidates, n_needed, replace=False))
    
    return unlensed_df.loc[sampled_indices]

unlensed_gals = create_2d_matched_sample(lrgs, lensed_gals, n_sample=10*len(z_lensed))

In [ ]:
_ = plt.hist(unlensed_gals['z_phot_median'], bins=1000)
_ = plt.hist(lensed_gals['z_phot_median'], bins=100)

In [ ]:
plt.plot(unlensed_gals['g_r'], unlensed_gals['dered_mag_r'], '.', label='unlensed')
plt.plot(lensed_gals['g_r'], lensed_gals['dered_mag_r'], '.', label='lensed')
plt.xlabel("Color (g-r)")
plt.ylabel("Magnitude (R-band)")
plt.legend()

# How many galaxies are left

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['SPS_HOME'] = "/global/homes/e/eramey16/fsps"
from docker import classify
from docker import db_util as util

import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
lrg_file = os.path.expandvars("$SCRATCH/data/monocle/LRGs/unlensed_h5/unlensed_LRGs.txt")
lrg_file_2 = os.path.expandvars("$SCRATCH/data/monocle/LRGs/unlensed_h5/unlensed_LRGs_2.txt")
unlensed_lsids = pd.read_csv(lrg_file, names=['ls_id'])
# unlensed_gals['ls_id'].to_csv(lrg_file, header=None, index=False)

In [3]:
# lrgs = pd.read_csv(lrg_file)
classy = classify.Classifier()
unused_lsids = []
for ls_id in unlensed_lsids.ls_id:
    try:
        bkdata, tbldata = classy.get_galaxy(ls_id, tag='unlensed_LRG')
        if bkdata.stage[0]!=2: unused_lsids.append(ls_id)
    except:
        unused_lsids.append(ls_id)
print(len(unused_lsids))

lrg_commands = [str(ls_id) for ls_id in unused_lsids]
# lrg_commands = [stmt + str(ls_id) for ls_id in unused_lsids]
# lrg_commands = lrg_commands[100:114]
# lrg_commands = stmt + unused_lsids.ls_id.astype(str)
# num_files = int(len(lrg_commands)/25)
# idxs = np.linspace(0, len(lrg_commands), num_files+1).astype(int)
# for i in range(num_files):
#     short_commands = lrg_commands.iloc[idxs[i]:idxs[i+1]]
    # short_commands.to_csv(command_file+f"_{i}.sh", index=False, header=False)
# lrg_commands.to_csv(command_file, index=False, header=False)
# with open(command_file, 'w') as file:
#     # file.write(header+'\n'.join(lrg_commands))
#     file.write('\n'.join(lrg_commands))
with open(lrg_file_2, 'w') as file:
    file.write('\n'.join(lrg_commands))
print(lrg_commands[:5])

229
['10995490503000982', '10995473899849837', '10995493508230774', '10995475401425056', '10995491983595455']


In [4]:
# Write an ML training script for these (with RF and XGBoost)
# Run and troubleshoot full scene modeling notebook (even without GPU)

# Plotting XGBoost Results

In [1]:
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
from docker import db_util as util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# # Colors are already de-reddened
# engine = sa.create_engine(util.conn_string, poolclass=NullPool)
# with engine.connect() as conn:
#     gal_tbl = sa.Table('lrg_train', sa.MetaData(), autoload_with=engine)
#     stmt = sa.select(gal_tbl)
#     gal_data = pd.DataFrame(conn.execute(stmt))

In [22]:
x = pd.DataFrame({'a': [1, np.nan, np.nan], 'b': [4, np.nan, 6], 'c':[np.nan, np.nan, np.nan]})
x[x[['a', 'b']].isnull().sum(axis=1)<2].reset_index(drop=True)

,a,b,c
0,1.0,4.0,NaN
1,NaN,6.0,NaN


In [16]:
x.a<3

0    False
1     True
2    False
Name: a, dtype: bool